## EMIT and ENMAP data coverage maps

Output in geojson and tif.

In [1]:
import pystac_client

def query_emit(bbox= (112, -44., 154, -9.), 
               timerange="2024-01-01T00:00:00Z/2024-12-31T23:59:59Z",
               max_cloud=10):
    # LPCLOUD catalog
    catalog_url = 'https://cmr.earthdata.nasa.gov/cloudstac/LPCLOUD/'

    search_params = {
        "collections": ["EMITL2ARFL_001"],  # Specify the collection
        "bbox": bbox,      # Define the bounding box (swLon,swLat,neLon,neLat)
        "datetime": timerange,  # Date range
        "query": {
            "eo:cloud_cover": {"lt": max_cloud}  # Query parameter: cloud cover less than 10%
        }
    }

    # Query STAC catalogls
    catalog = pystac_client.Client.open(catalog_url)

    # Run the STAC query
    query = catalog.search(**search_params)

    return list(query.items())

In [7]:
def query_enmap(bbox= (112, -44., 154, -9.),
                timerange="2024-01-01T00:00:00Z/2024-12-31T23:59:59Z",
                max_cloud=10):
    """
    Query the STAC catalog with the specified bounding box and return matching items.

    :param bbox: List of [swLon, swLat, neLon, neLat] defining the bounding box.
    :return: ItemCollection returned by the query.
    """
    # Open the catalog
    catalog_url = "https://geoservice.dlr.de/eoc/ogc/stac/v1/"
    catalog = pystac_client.Client.open(catalog_url)

    # Collections to be searched
    collections = ["ENMAP_HSI_L2A"]

    # Perform the search
    search = catalog.search(
        collections=collections,
        bbox=bbox,
        datetime=timerange
    )

    # Return the items collection
    items = search.item_collection()

    keep_items=[]
    for item in items:
        cloud_cover = float(item.properties.get("eo:cloud_cover", 100))  # Default to 100 if not available
        biome_type = item.properties.get("enmap:biomeType", 'not available')
        if cloud_cover < max_cloud and biome_type != 'not available':
            keep_items.append(item)

    return keep_items



In [3]:
from datetime import datetime
import json

def parse_datetime(datetime_str):
    # Try to parse datetime with and without microseconds
    try:
        return datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    except ValueError:
        return datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%SZ")
    
def create_geojson_feature_collection(items, output, sensor="emit"):
    features = []

    for item in items:
        
        emit_feature = {
            "type": "Feature",
            "geometry": item.geometry,
            "properties": {
                "id": item.id,
                "datetime": parse_datetime(item.properties["datetime"]).strftime("%Y-%m-%d %H:%M:%S"),
                "cloud_cover": float(item.properties["eo:cloud_cover"]),
                "type": sensor,
            }
        }

        features.append(emit_feature)

    feature_collection = {
        "type": "FeatureCollection",
        "features": features
    }

    with open(output, 'w') as f:
        json.dump(feature_collection, f, indent=2)

    return feature_collection

In [4]:
import geopandas as gpd
import rasterio
import numpy as np
from rasterio import features
from affine import Affine
import pandas as pd


def save_to_raster(features_file):
    gdf = gpd.read_file(features_file)  # or use gpd.GeoDataFrame for in-memory features

    # Define output resolution and bounds
    resolution = 0.001  # degrees per pixel
    minx, miny, maxx, maxy = gdf.total_bounds
    width = int((maxx - minx) / resolution)
    height = int((maxy - miny) / resolution)

    # Define transform (affine)
    transform = Affine.translation(minx, maxy) * Affine.scale(resolution, -resolution)

    # Assume you want to count how many times a pixel is covered by any geometry
    raster = np.zeros((height, width), dtype="uint8")

    gdf['datetime'] = pd.to_datetime(gdf['datetime'])

    # Group by date (no time)
    gdf['date'] = gdf['datetime'].dt.date
    grouped = gdf.groupby('date')
    for date, group in grouped:
        layer = features.rasterize(
            ((geom, 1) for geom in group.geometry),
            out_shape=(height, width),
            transform=transform,
            fill=0,
            dtype="uint8"
        )
        raster += layer  # count overlaps

    output = features_file.replace('.geojson','.tif')
    # Save to GeoTIFF
    with rasterio.open(
        output,
        "w",
        driver="GTiff",
        height=raster.shape[0],
        width=raster.shape[1],
        count=1,
        dtype=raster.dtype,
        crs=gdf.crs.to_string(),
        transform=transform,
    ) as dst:
        dst.write(raster, 1)



In [10]:
year = 2024
max_cloud = 10

items= query_emit(timerange=f"{year}-01-01T00:00:00Z/{year}-12-31T23:59:59Z",
               max_cloud=max_cloud)
scenes = create_geojson_feature_collection(items, output =f'emit_features_{year}_maxcloud{max_cloud}.geojson')
save_to_raster(features_file=f'emit_features_{year}_maxcloud{max_cloud}.geojson')

In [9]:
year = 2024
max_cloud = 10

items= query_enmap(timerange=f"{year}-01-01T00:00:00Z/{year}-12-31T23:59:59Z",
               max_cloud=max_cloud)
scenes = create_geojson_feature_collection(items, output =f'enmap_features_{year}_maxcloud{max_cloud}.geojson', sensor='enmap')
save_to_raster(features_file=f'enmap_features_{year}_maxcloud{max_cloud}.geojson')